In [8]:
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.stats import sigma_clipped_stats

In [9]:
# searching L 98-59
Target = "TIC 307210830"
search = lk.search_targetpixelfile(Target, mission="TESS")
print (search)

SearchResult containing 66 data products.

 #     mission     year   author  exptime target_name distance
                                     s                 arcsec 
--- -------------- ---- --------- ------- ----------- --------
  0 TESS Sector 02 2018      SPOC     120   307210830      0.0
  1 TESS Sector 05 2018      SPOC     120   307210830      0.0
  2 TESS Sector 12 2019      SPOC     120   307210830      0.0
  3 TESS Sector 11 2019      SPOC     120   307210830      0.0
  4 TESS Sector 10 2019      SPOC     120   307210830      0.0
  5 TESS Sector 09 2019      SPOC     120   307210830      0.0
  6 TESS Sector 08 2019      SPOC     120   307210830      0.0
  7 TESS Sector 29 2020      SPOC      20   307210830      0.0
  8 TESS Sector 28 2020      SPOC      20   307210830      0.0
  9 TESS Sector 32 2020      SPOC      20   307210830      0.0
 10 TESS Sector 32 2020      SPOC     120   307210830      0.0
 11 TESS Sector 29 2020      SPOC     120   307210830      0.0
 12 TESS Sec

In [10]:
all_tpfs = search.download_all()
print(f"Downloaded {len(all_tpfs)} TPF(s)")

LightkurveError: Unexpected error in reading data product: 'TypeError: buffer is too small for requested array'
C:\Users\Coral14\.lightkurve\cache\mastDownload\TESS\tess2025014115807-s0088-0000000307210830-0285-a_fast\tess2025014115807-s0088-0000000307210830-0285-a_fast-tp.fits
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.

In [11]:
# extracting aperture-based lightcurves (from on tpf now)
def lcs(all_tpfs):
    lightcurves = {}
    # SPOC
    lightcurves['pipeline'] = all_tpfs.to_lightcurve(aperture_mask = all_tpfs.pipeline_mask)
    # masks by size, using lk helpers
    lightcurves['small'] = all_tpfs.to_lightcuve(apperture_mask='small')
    lightcurves['medium'] = all_tpfs.to_lightcuve(apperture_mask='medium')
    lightcurves['large'] = all_tpfs.to_lightcuve(apperture_mask='large')
    #custom threshold mask
    mask = all_tpfs.create_threshold_mask(threshold = 10)
    lightcurves['custom_thres10'] = all_tpfs.to_lightcurve(aperture_mask = mask)
    return lightcurves

In [12]:
# building combined list of possible lcs from all_tpfs (per sector)
possible = []
for tpf in all_tpfs:
    lightcurves = lcs(all_tpfs)
    for name, lc in lightcurves.items():
        #remove NaNs and normalize
        lc = lc.remove_nans().normalize()
        possible.append({'sector': tpf.sector, 'mask': name, 'lightcurve': lc})

NameError: name 'all_tpfs' is not defined

In [14]:
# calc noise for each possible, use: estimate_cdpp & simple RMS
def reduce_noise_for_lc(lc):
    # use estimate_cdpp (built-in noise metric)
    try:
        cdpp = lc.estiamte_cdpp() # returns float
        # if return obj, take value
        if hasattf(cdpp, 'value'):
            cdpp_value = float(cdpp.value)
        else:
            cdpp_value = float(cdpp)
    except Exception:
        cdpp_value = np.nan
    # calc RMS
    flux = lc.flux.value
    median = np.nanmedian(flux)
    rms = np.nanstd(flux - median)
    return cdpp_value, rms

        

In [15]:
# evaluate all possible curves and rank to determine best
results = []
for c in possible:
    cdpp_value, rms = reduce_noise_for_lc(c['lc'])
    reults.append({'sector': c['sector'], 'mask': c['mask'], 'cdpp': cdpp_value, 'RMS': rms, 'lightcurve': c['lc']})


In [19]:
# turn into sorted list (sorted via cdpp), we want lower, then rms
sorted_results = sorted(results, key=lambda x: (np.nan_to_num(x['cdpp'], nan=1e9)))
for r in sorted_results[:10]:
    print(r['sector'], r['mask'], "CDPP:", r['cdpp'], "RMS:", r['rms'])
                            

In [20]:
# pick best lightcurve
best = sorted_results[0]['lc']
best.plot()
plt.title(f"Best lightcurve: sector {sorted_results[0]['sector']} mask {sorted_results[0]['mask']}")

IndexError: list index out of range

In [21]:
# create periodogram with lomb-scargle, find top power peak
pg = best.to_periodogram(method='lombscargle', minimimum_period=0.1, maximum_period=50) # can adjust bounds as needed
period = pg.period_at_max_power
print("Best period (peak):", period)

NameError: name 'best' is not defined

In [22]:
# fold lightcurve using best period + plot
folded_lc = best.fold(period.value)
folded_lc.scatter()
plt.gca().invert_yaxis()
plt.title(F"Folded on P = {period:.6f} days")


NameError: name 'best' is not defined

In [23]:
import os
os.getcwd()

'C:\\Users\\Coral14\\Documents\\TESS-Exoplanet\\tess-env'

In [25]:
import shutil
shutil.move("Testing.ipynb", "/Users/Coral14/Documents/TESS-Exoplanets/tess-env")

FileNotFoundError: [WinError 3] The system cannot find the path specified